# A 3-layer neural network

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [2]:
#Read data
data = pd.read_pickle('./alldata.txt')
data.head()

#normalization
X = data.drop(['Label'], axis=1)
X_data = (X - X.min())/(X.max() - X.min())
X_data = X_data.drop(X_data.columns[X_data.isna().any().tolist()], axis=1)
# X_data.head()
X_data = np.float32(X_data)
y_data = 1 - np.float32(data['Label'] == 'BENIGN') #y = 1 represents anomalies

In [3]:
with tf.Session() as sess:
    y_data = sess.run(tf.one_hot(y_data, depth=2))

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

In [27]:
ninput = 70
nlayer1 = 256
nlayer2 = 256
learning_rate = 0.0001
num_steps = 30000
display_step = 1000
batch_size = 256
nclasses = 2
# nlayer3 = 256

X = tf.placeholder(tf.float32, shape=[None, ninput], name='X')
y = tf.placeholder(tf.float32, shape=[None, nclasses], name='y')
W1 = tf.Variable(tf.random_normal([ninput, nlayer1]))
W2 = tf.Variable(tf.random_normal([nlayer1, nclasses]))
# W3 = tf.Variable(tf.random_normal([nlayer2, nclasses]))
b1 = tf.Variable(tf.random_normal([nlayer1]))
b2 = tf.Variable(tf.random_normal([nclasses]))
# b3 = tf.Variable(tf.random_normal([nclasses]))
# b3 = tf.Variable(tf.random_normal())

layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
# layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
output = tf.matmul(layer1, W2) + b2
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [28]:
X_train.shape

(1981520, 70)

In [29]:
cnt = 2
X_train[(cnt - 1) * batch_size : cnt * batch_size, :].shape

(256, 70)

In [30]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #     sess.run(iterator.initializer)
    cnt = 0
    X_train, y_train = shuffle(X_train, y_train)
    cntMax = X_train.shape[0]/batch_size
    X_test, y_test = shuffle(X_test, y_test)
    x_index = []
    train_loss = []
    test_loss = []
    for i in range(1, num_steps+1):
        cnt = cnt + 1
        if cnt > cntMax:
            X_train, y_train = shuffle(X_train, y_train)
            cnt = 1
        _, l = sess.run([optimizer, loss], 
                        feed_dict={X: X_train[(cnt - 1) * batch_size : cnt * batch_size, :], 
                                   y: y_train[(cnt - 1) * batch_size : cnt * batch_size, :]})
        # Display logs per step
#         ntest = 0
        if i % display_step == 0: # or i == 1:
#             ntest += 1;
#             x_index.append(i)
#             train_loss.append(l)
#             l_test = sess.run([loss],
#                               feed_dict={X: X_test[(ntest - 1) * batch_size : ntest * batch_size, :],
#                                   y: y_test[(ntest - 1) * batch_size : ntest * batch_size]})
#             test_loss.append(l_test[0])
#           
            print('Step %i: Minibatch Loss: %f' % (i, l))
    
    #Calculate the accuracy over whole set
    y_train_pred = sess.run(output, feed_dict={X: X_train, 
                               y: y_train})
    y_test_pred = sess.run(output, feed_dict={X: X_test, 
                                              y: y_test})
#     print(train_pred)
#    

Step 1000: Minibatch Loss: 0.563367
Step 2000: Minibatch Loss: 0.195919
Step 3000: Minibatch Loss: 0.177380
Step 4000: Minibatch Loss: 0.100634
Step 5000: Minibatch Loss: 0.080668
Step 6000: Minibatch Loss: 0.096754
Step 7000: Minibatch Loss: 0.094645
Step 8000: Minibatch Loss: 0.070522
Step 9000: Minibatch Loss: 0.125488
Step 10000: Minibatch Loss: 0.099386
Step 11000: Minibatch Loss: 0.100657
Step 12000: Minibatch Loss: 0.077425
Step 13000: Minibatch Loss: 0.066098
Step 14000: Minibatch Loss: 0.060641
Step 15000: Minibatch Loss: 0.091194
Step 16000: Minibatch Loss: 0.102623
Step 17000: Minibatch Loss: 0.145610
Step 18000: Minibatch Loss: 0.065760
Step 19000: Minibatch Loss: 0.090371
Step 20000: Minibatch Loss: 0.075815
Step 21000: Minibatch Loss: 0.071890
Step 22000: Minibatch Loss: 0.071415
Step 23000: Minibatch Loss: 0.070838
Step 24000: Minibatch Loss: 0.072046
Step 25000: Minibatch Loss: 0.054004
Step 26000: Minibatch Loss: 0.091034
Step 27000: Minibatch Loss: 0.064846
Step 28000

In [75]:
def TP(y1, y2): #y1 = y; y2 = y_pred
    return ((y1.argmax(axis=1) == 0) & (y2.argmax(axis=1) == 0)).sum()
def FN(y1, y2):
    return ((y1.argmax(axis=1) == 0) & (y2.argmax(axis=1) == 1)).sum()
def FP(y1, y2):
    return ((y1.argmax(axis=1) == 1) & (y2.argmax(axis=1) == 0)).sum()
def TN(y1, y2):
    return ((y1.argmax(axis=1) == 1) & (y2.argmax(axis=1) == 1)).sum()
    
def confusionMatrix(y1, y2):
    return np.array([[TP(y1, y2), FN(y1, y2)], [FP(y1, y2), TN(y1, y2)]])

In [78]:
def accuracy(mat):
    return (mat[0][0] + mat[1][1])/mat.sum()
def precision(mat):
    return mat[0][0]/(mat[0][0] + mat[1][0])
def recall(mat):
    return mat[0][0]/(mat[0][0] + mat[0][1])

def show_result(mat):
    print('Accuracy = ' + str(accuracy(mat)) )
    print('Precision = ' + str(precision(mat)) )
    print('Recall = ' + str(recall(mat)) )

In [82]:
#Confusion matrix of train set
train_CM = confusionMatrix(y_train, y_train_pred) 
train_CM

array([[1569541,   21509],
       [  42685,  347785]])

In [83]:
show_result(train_CM)

Accuracy = 0.967603657798054
Precision = 0.9735241833340983
Recall = 0.986481254517457


In [84]:
#Confusion matrix of test set
test_CM = confusionMatrix(y_test, y_test_pred) 
test_CM

array([[672706,   9341],
       [ 18190, 148986]])

In [85]:
show_result(test_CM)

Accuracy = 0.9675809534127079
Precision = 0.9736718695722656
Recall = 0.9863044628889212
